In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
import math
import shutil
import os, sys

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
workspace = '/content/drive/MyDrive/Colab Notebooks/Text2Struct/'
print(os.path.exists(workspace))

modules_dir = workspace + 'modules/'
sys.path.insert(0, modules_dir)

dataset_dir = workspace + 'dataset/'
pubmed_dir = dataset_dir + 'pubmed/'
segmented_dir = dataset_dir + 'segmented/'

True


In [ ]:
if not os.path.exists(segmented_dir):
  os.makedirs(segmented_dir)

In [ ]:
df = pd.read_csv(pubmed_dir + 'pmid-thrombecto-set 2021-2022.csv')
print(len(df))
df_pm = df[df['Abstract'].notna()]
df_pm['PMID']

2451


0       34785611.0
1       34785610.0
2       34774198.0
3       35172028.0
4       34816417.0
           ...    
2446    36342173.0
2447    33722960.0
2448    34750282.0
2449    35210331.0
2450    33771936.0
Name: PMID, Length: 2262, dtype: float64

In [ ]:
def remove_empty_folders(path_abs):
    walk = list(os.walk(path_abs))
    for path, _, _ in walk[::-1]:
        if len(os.listdir(path)) == 0:
            os.rmdir(path) 

In [ ]:
def check_numeral(string):
  try:
      float(string)
      return True
  except ValueError:
      return False

In [ ]:
def sentence_proc(string):
  tokens = nltk.word_tokenize(string)

  for i in range(len(tokens)-1):
    # if re.sub(r"[^0-9.-]", "", tokens[i]) != '' and tokens[i+1] == '%':
    if check_numeral(tokens[i]) and tokens[i+1] == '%':
      tokens[i] = str(round(float(re.sub(r"[^0-9.-]", "", tokens[i])) * 0.01, 3))
      tokens[i+1] = ''

  tokens = [token for token in tokens if token != '']  

  string_new = ''
  for token in tokens:
    # if re.sub(r"[^0-9.-]", "", token) != '':
    #   token = re.sub(r"[^0-9^a-z^A-Z.-]", "", token)

    if re.search("\d,\d", token):
      token = re.sub(",", "", token) 

    if re.search("\d-\d", token):
      token = re.sub("-", " - ", token) 

    if re.search("\d\\+\d", token):
      token = re.sub("\\+", " + ", token)

    if re.search("\d\\+\\/-\d", token):
      token = re.sub("\\+\\/-", " +/- ", token) 

    if re.search("\d\\/\d", token):
      token = re.sub("\\/", " / ", token) 

    if re.search("=", token):
      token = re.sub("=", " = ", token)
    
    string_new += ' ' + token

  string_new = string_new[1:]
  string_new = re.sub(' +', ' ', string_new)
  
  return string_new

In [ ]:
## Checking
string = "This is a test of patterns: p2, 2b-10, 50,000, 98%, 9-10, 9+10, 9.5+/-5, p=0.05, [90.89], 80-90%"
sentence_proc(string)

'This is a test of patterns : p2 , 2b-10 , 50000 , 0.98 , 9 - 10 , 9 + 10 , 9.5 +/- 5 , p = 0.05 , [ 90.89 ] , 80 - 90 %'

In [ ]:
N_abs = 100
ann_config_src = dataset_dir + 'annotation.conf'

for i in range(N_abs):
  pmid_abs = int(df_pm['PMID'].iloc[i])
  text_abs = df_pm['Abstract'].iloc[i]

  dir_abs = segmented_dir + 'PMID_' + str(pmid_abs) + '/'
  numbers_abs = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", text_abs)
  if (not os.path.exists(dir_abs)) and len(numbers_abs) > 0:
    os.makedirs(dir_abs)

  sentences = nltk.tokenize.sent_tokenize(text_abs)
  for (j, sentence) in enumerate(sentences):
    numbers_sen = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", sentence)
    file_sen = dir_abs + 'S_' + str(j) + '.txt'
    if (not os.path.exists(file_sen)) and len(numbers_sen) > 0:
      with open(file_sen, 'w') as f:
        f.write(sentence_proc(sentence))

  ann_config_dst = dir_abs + 'annotation.conf'
  if os.path.exists(dir_abs) and (not os.path.exists(ann_config_dst)):
    shutil.copyfile(ann_config_src, ann_config_dst)